# Génération de musique par Intelligence Artificielle pour la pratique sportive 

## Partie 3: Entraînement/affinage (*"spécialisation"*) d'un modèle d'IA Générative

Les 2 cellules ci-dessous chargent quelques modules utilitaires que nous ne détaillerons pas ici.

In [ ]:
%run utilitaires.ipynb

In [ ]:
%load_ext autoreload
%autoreload 2

### Chargement des données

Nous devons tout d'abord charger le jeux de données que nous avons préparé (notebook 2).

In [ ]:
data_path = "./data/music/processed/all"
dataset = load_from_disk(data_path)

dataset

### Chargement du modèle MusicGen

Comme dans le notebook 1 dédié au prompt engineering et au zero-shot learning, nous devons charger l'algorithme d'IA (toujours en 2 parties: le `processor` et le `model` lui-même).

In [ ]:
processor = AutoProcessor.from_pretrained(
    "facebook/musicgen-medium",
    cache_dir="./models"
)

model = AutoModelForTextToWaveform.from_pretrained(
    "facebook/musicgen-medium",
    cache_dir="./models"
)

# The following configurations are needed to avoid a bug while training:
model.config.decoder_start_token_id = model.decoder.config.bos_token_id
model.config.decoder.decoder_start_token_id = model.decoder.config.bos_token_id

model.freeze_text_encoder()
model.freeze_audio_encoder()

torch.cuda.empty_cache()

print(model)
print(processor)

### Définition d'un sous-ensemble des paramètres à ajuster

Le modèle contient un grand nombre de paramètres (~2 milliards). C'est relativement petit par rapport aux modèles existants (ChatGPT, Gemini, Grok, etc...), mais c'est beaucoup pour les ressources à notre disposition. 

Calculer toutes les dérivées (et plus précisément les [gradients](https://fr.wikipedia.org/wiki/Gradient#:~:text=En%20physique%20et%20en%20analyse,employa%20avant%20les%20autres%20disciplines)) par rapport à tous les paramètres serait trop coûteux en calcul et mémoire.

De plus, notre jeux de données est bien trop petit par rapport aux nombres de paramètres, il y a donc un risque de [surapprentissage](https://fr.wikipedia.org/wiki/Surapprentissage#:~:text=En%20statistique%2C%20le%20surapprentissage%2C%20ou,d'un%20ensemble%20de%20donn%C3%A9es.).

Pour toutes ces raisons nous allons uniquement ajuster un petit nombre de paramètres.

In [ ]:
target_modules = (
    [
        "enc_to_dec_proj",
        "audio_enc_to_dec_proj",
        "k_proj",
        "v_proj",
        "q_proj",
        "out_proj",
        "fc1",
        "fc2",
        "lm_heads.0",
    ]
    + [f"lm_heads.{str(i)}" for i in range(len(model.decoder.lm_heads))]
    + [f"embed_tokens.{str(i)}" for i in range(len(model.decoder.lm_heads))]
)

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
)
# model.enable_input_require_grads()
# model.disable_input_require_grads()
model = get_peft_model(model, config)
model.print_trainable_parameters()

### Entraînement (fine-tuning) du modèle d'Intelligence Artificielle

Nous voilà désormais prêts à affiner MusicGen avec notre nouveau jeux de données.

Observez l'évolution du `Training loss`: c'est ce qui mesure la "distance" entre les prédictions faites par MusicGen et les musiques de notre jeux de données. Plus cette quantité diminue, plus les prédictions de MusicGen sont sensées se rapprocher des musiques fournies en entrée. Bien évidemment, la notion de distances entre deux musiques est plus complexe qu'une simple distance "euclidienne".

Le `Training loss` va avoir tendance à diminuer au cours du temps, mais il n'est pas forcément strictement décroissant (il peut augmenter avant de diminuer, etc...).

In [ ]:
torch.cuda.empty_cache()

training_args = Seq2SeqTrainingArguments(
      output_dir="./tmp/output/",
      num_train_epochs=5,
      # gradient_accumulation_steps=8,
      gradient_checkpointing=True,  # True,
      per_device_train_batch_size=8,  # 2,
      learning_rate=5e-5,  # 2e-4,
      weight_decay=0.1,  # 0.1,
      adam_beta2=0.99,  #0.99,
      # fp16=True,
      dataloader_num_workers=15,
      logging_steps=1,
      # report_to="none",
      disable_tqdm=False,
)

# model.config.decoder_start_token_id = model.decoder.config.bos_token_id


# Initialize MusicgenTrainer
trainer = MusicgenTrainer(  # Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorMusicGenWithPadding(
        processor=processor,
    ),
    train_dataset=dataset.remove_columns(["attention_mask", "audio", "description"]).shuffle(seed=17),
    # processing_class=processor.__class__,
    # tokenizer=processor,
)

train_result = trainer.train()

### Sauvegarde du modèle spécialisé

In [ ]:
save_path = "./models/fine_tuned_musigen_all"

os.makedirs(save_path, exist_ok=True)
shutil.rmtree(save_path)

model.save_pretrained(save_path)